# Variables climáticas anuales a partir de climáticas diarias

Este archivo es importante para pasar de 19 filas (19 distritos) del df_0 a 19 (ditritos) x 13 (años) = 247 filas.
Ya tenemos datos de incendios por distrito entre el 2007 y el 2019 incluidos
En este df_1 asumimos que las variables ecológicas y de superficie se mantienen constantes.
Naturalmente, al darle más variabilidad climática le dará más peso al clima y menos a las variables específicas. Aún así es interesante.

Tenemos descargados todos los csv para todos los distritos, pluviosidad diaria entre el 2007 y el 2019, excepto para el distrito 5 que tenemos desde el 2010 al 2019. Hemos tomado datos entre el 10 de junio y el 10 noviembre, que es la temporada en la que las sequías pueden causar incendios.
Los archivos están en la carpeta "climati_csv" alojada en la misma que el codigo Los csv están nombrados de forma sistemática: distrit01_plu_ju07-no11: pluviosidad en distrito 1 entre el 10 junio del 2007 y el 10 noviembre del 2011. A ver que se hace con el distrito 5: imputar o no usar los años faltantes.

Comenzaremos por descargar los datos de incendios y los ecológicos (porcetange de ocupación por especia para cada distrito)



In [30]:
# Hectareas quemadas por año y distrito:

import numpy as np
import pandas as pd

# df_ha_qmd = pd.read_excel("anho_distrito_has_quemad.xls")
# df_ha_qmd.head()


Como lo que queremos es tener muchas filas, crearemos una fila que sea año donde meter a las demas 

In [31]:
# df_ha_qmd.stack()

In [32]:
# Esto no va muy bien
# Por lo que modifiqué el excel para no corregirlo

df_ha_qmd = pd.read_excel("colum_quem_anh.xls")
df_ha_qmd

,num_distrito,anho,ha_quemadas
0,1,2007,106.80
1,2,2007,126.30
2,3,2007,185.40
3,4,2007,29.90
4,5,2007,610.60
...,...,...,...
242,15,2019,333.43
243,16,2019,262.24
244,17,2019,68.51
245,18,2019,33.57


In [33]:
# Porcentaje de ocupación por grupo de especies por año y distrito:
# coníferas, eucaliptos, caducifolias

df_pc_sps = pd.read_excel("variab_ecol_anh_distrit.xls")
df_pc_sps.head(5)


,DISTRITO,num_distr,año,ha_man_comun,ha_monte_particular,% ocup medio coniferas,% ocup medio frond_peren,% ocup medio cadicifolias,% ocup hipotetico coniferas,% ocup hipotetico frond_peren,% ocup hipotetico cadicifolias
0,Ferrol,1,2007,4952.62,85072,5,65,10,5,5,70
1,Bergantiños - Mariñas Coruñesas,2,2007,3145.99,90980,27,28,16,3,3,65
2,Santiago - Meseta Interior,3,2007,3919.24,96731,22,32,24,4,4,70
3,Barbanza,4,2007,24703.18,73426,33,29,9,2,2,68
4,Fisterra,5,2007,11350.17,79115,15,18,8,2,2,37


In [34]:
df_ha_qmd.columns

Index(['num_distrito', 'anho', 'ha_quemadas'], dtype='object')

In [35]:
# Ahora unimos los dos df

# df_qe_es = pd.concat([df_pc_sps, df_ha_qmd], axis=1)
# Mejor con un merge,  que usa columnas (join indice)

# df_ha_qmd.set_index(['num_distrito', 'anho']) (para hacer un join)
df_qe_es = df_pc_sps.merge(df_ha_qmd, how='inner', left_on=['num_distr', 'año'], right_on=['num_distrito', 'anho'])

In [36]:
# borro anho y distrito (repes)

df_qe_es = df_qe_es.drop(['num_distrito', 'anho'], axis = 1)

# df_qe_es = df_qe_es.drop(["num_distrito"], axis = 1)  
# Al intentar borrar la columa num distrito da error:
# KeyError: "['num_distrito'] not found in axis"

df_qe_es.head(5)

,DISTRITO,num_distr,año,ha_man_comun,ha_monte_particular,% ocup medio coniferas,% ocup medio frond_peren,% ocup medio cadicifolias,% ocup hipotetico coniferas,% ocup hipotetico frond_peren,% ocup hipotetico cadicifolias,ha_quemadas
0,Ferrol,1,2007,4952.62,85072,5,65,10,5,5,70,106.8
1,Bergantiños - Mariñas Coruñesas,2,2007,3145.99,90980,27,28,16,3,3,65,126.3
2,Santiago - Meseta Interior,3,2007,3919.24,96731,22,32,24,4,4,70,185.4
3,Barbanza,4,2007,24703.18,73426,33,29,9,2,2,68,29.9
4,Fisterra,5,2007,11350.17,79115,15,18,8,2,2,37,610.6


Puede ser interesante crear una columan de hectáreas totales y otra de porcentage de hectareas quemadas, antes de escalar.

In [37]:
# Crear esas columnas

df_qe_es["Ha_tot"] = (df_qe_es["ha_man_comun"] + df_qe_es["ha_monte_particular"])
df_qe_es["Prop_quem"] = (df_qe_es["ha_quemadas"] / df_qe_es["Ha_tot"])
df_qe_es

,DISTRITO,num_distr,año,ha_man_comun,ha_monte_particular,% ocup medio coniferas,% ocup medio frond_peren,% ocup medio cadicifolias,% ocup hipotetico coniferas,% ocup hipotetico frond_peren,% ocup hipotetico cadicifolias,ha_quemadas,Ha_tot,Prop_quem
0,Ferrol,1,2007,4952.62,85072,5,65,10,5,5,70,106.80,90024.62,0.001186
1,Bergantiños - Mariñas Coruñesas,2,2007,3145.99,90980,27,28,16,3,3,65,126.30,94125.99,0.001342
2,Santiago - Meseta Interior,3,2007,3919.24,96731,22,32,24,4,4,70,185.40,100650.24,0.001842
3,Barbanza,4,2007,24703.18,73426,33,29,9,2,2,68,29.90,98129.18,0.000305
4,Fisterra,5,2007,11350.17,79115,15,18,8,2,2,37,610.60,90465.17,0.006750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,A Limia,15,2019,58467.10,52163,20,2,21,4,2,37,333.43,110630.10,0.003014
243,Deza - Tabeirós,16,2019,16179.94,44372,24,12,34,5,5,50,262.24,60551.94,0.004331
244,O Condado - A Paradanta,17,2019,31011.84,34448,28,42,17,6,6,75,68.51,65459.84,0.001047
245,Vigo - Baixo Miño,18,2019,37737.61,33827,19,34,13,4,4,58,33.57,71564.61,0.000469


# Vamos ahora a por los datos climáticos 

Queremos que devuelva, para cada año entre el 10 de junio y el 10 de noviembre:

a) un número máximo de días seguidos sin llover (los incendios vienen tras sequías). 
De ser posible se puede añadir alguna columna más con los siguientes puestos en el ranking de días seguidos sin llover para un determinado medio-año.

Esto es lo dificil de resolver. Las series temporales de panda no ayudan mucho, habrá que crear un bucle probablemente.

b) número total de días sin llover (también ayudará a predecir incendios en años sin grandes secas) 
Un simple count when.

c) y por qué no, una pluviosidad anual (habría que probar con o sin ella, ya que puede confundir) 
Un count.



In [38]:
import pandas as pd
import os

DATOS_DIR = os.path.join(os.getcwd(), 'climati_csv')

def load_dataframe(file):
    distrito = int(file[7:9])
    df = pd.read_csv(os.path.join(DATOS_DIR, file))
    df['distrito'] = distrito
    return df

df = pd.concat([
    load_dataframe(x) for x in os.listdir(DATOS_DIR)
])

df = df.sort_values(by=['distrito', 'Instante lectura']).reset_index(drop = True)
df

,Instante lectura,Chuvia,distrito
0,2007-06-10 00:00:00.0,0.2,1
1,2007-06-11 00:00:00.0,0.2,1
2,2007-06-12 00:00:00.0,2.0,1
3,2007-06-13 00:00:00.0,13.0,1
4,2007-06-14 00:00:00.0,5.0,1
...,...,...,...
76472,2019-11-06 00:00:00.0,8.8,19
76473,2019-11-07 00:00:00.0,4.7,19
76474,2019-11-08 00:00:00.0,7.4,19
76475,2019-11-09 00:00:00.0,10.8,19


In [39]:
df.dtypes

Instante lectura     object
Chuvia              float64
distrito              int64
dtype: object

In [40]:
# Vamos a limpiar un poco, para quedarnos solo con datos del 10-6 al 10-11 incluidos.
# Primero pasamos a fecha

df['Instante lectura'] = pd.to_datetime(df['Instante lectura'])

In [41]:
# Con este código nos quedamos para cada año con los días entre
# el 10 de junio y el 10 de noviembre

df_meses_completos = df[df['Instante lectura'].dt.month.isin([7, 8, 9, 10])]
df_junio = df[(df['Instante lectura'].dt.month == 6) & (df['Instante lectura'].dt.day >= 10)]
df_noviembre = df[(df['Instante lectura'].dt.month == 11) & (df['Instante lectura'].dt.day <= 10)]
df = pd.concat([df_junio, df_meses_completos, df_noviembre])


In [42]:
df.shape

(37252, 3)

Bien, tenemos pues 37.252 filas.

¿Cúantas deberíamos tener?

Son registros diariso de 13 años.
Tendresmo 154 días por año: entre 10 de junio y 10 de noviembre.


21 del 10 al 30 de junio (incluidos), más 10 de noviembre: 31
31 x 3 de julio, agosto, y octubre, y 30 septiembre: 123. 
total: 123 + 31 = 154

Tenemos estos datos para 19 distritos: 13 x 154 = 2002
2002 x 19 = 38038 . 

Ahora bien, a ésto hay que restarle 3 años (2007, 2008, 2009) que faltan del distrito 5:
154 x 3 = 462.  Es decir, esperamos obtener un df con 38038 - 462 = 37576 filas. 
Tenemos 37252 y deberíamos tener 37576. 

Es decir, faltan 324 registros en total.
Parece mucho, aunque en realidad es menos de un 1% del total de registros diarios.

Es probable que se haya descargado algún archivo con muchos datos faltantes.
Lo veremos con una función pivot



In [43]:
# Lo primero es crear una columna llamada año

df['year'] = df['Instante lectura'].dt.year
df.head()

,Instante lectura,Chuvia,distrito,year
0,2007-06-10,0.2,1,2007
1,2007-06-11,0.2,1,2007
2,2007-06-12,2.0,1,2007
3,2007-06-13,13.0,1,2007
4,2007-06-14,5.0,1,2007


In [44]:
# Hagamos un pivot para ver todos

# df.pivot(index='distrito', columns='anho')['chuvia'].count()   #falla
# df.pivot(index='distrito', columns='anho', values='Chuvia')   # también falla

c_df = df[['distrito', 'year', 'Chuvia']]

print(pd.pivot_table(c_df,
           index='distrito',
           columns='year',
           margins=True,
           aggfunc=len))


          Chuvia                                                          \
year        2007    2008    2009    2010    2011    2012    2013    2014   
distrito                                                                   
1          154.0   154.0   154.0   154.0   154.0   154.0   154.0   154.0   
2          154.0   154.0   154.0   154.0   154.0   154.0   154.0   154.0   
3          154.0   154.0   154.0   154.0   154.0   154.0   154.0   154.0   
4          154.0   154.0   154.0   154.0   154.0   154.0   154.0   154.0   
5            NaN     NaN     NaN   154.0   154.0   154.0   154.0   154.0   
6          154.0   154.0   154.0   154.0   154.0   154.0   154.0   154.0   
7          154.0   154.0   154.0   154.0   154.0   154.0   154.0   151.0   
8          152.0   154.0   154.0   154.0   154.0   154.0   154.0   154.0   
9          154.0   154.0   154.0   154.0   154.0   154.0   154.0   154.0   
10         153.0   154.0   154.0   154.0   154.0   154.0   154.0   154.0   
11         1

Vemos que prácticamente cada conjunto año-distrito tiene 154 días, con excepción de:
los años 2017 y 2018 del distrito 15
los años 2007, 2008, 2009 del distrito 5

volvamos a las cuentas: tenemos  37.252 filas sobre las 38038  que deberíamos tener

Nos faltan datos de cinco años-distrito : 5 x 154 = 770 deberían faltar

y por otro lado 38038 (debería haber) - 37252 (hay) = 786 faltan

Es decir, faltan 16 filas más de las que deberían faltar 

Es un misterio.

Veamos si hay huecos entre las filas (código cortesía de Roi)

In [45]:
# Para ver si hay huecos subimos la fecha un día y lo restamos al original.
# Si la diferencia es 1 no habrá huecos en ese día

from datetime import timedelta

def groupby_diff(inner_df):
    test = sum(~(inner_df['Instante lectura'] - inner_df['Instante lectura'].shift(1) == timedelta(days=1))) - 1
    return test

huecos = df.groupby(by=['distrito', 'year']).apply(groupby_diff)
huecos[huecos > 0]

# Lo que estamos haciendo aquí es restar las fechas originales a fechas atrasadas
# Si hay huecos en las fechas este valor es distinto de 1
# Conceptos avanzados: grooupbydiff, inner_df, timedelta.

distrito  year
7         2014    3
8         2007    1
          2018    3
10        2007    1
          2018    3
dtype: int64

Vemos que hay años que tienen sus huecos. 
distrito 7-2017, 8-2007, 8-20018, 10-2007, 10-2018
Lo raro es que todos esos años-distrito tienen 154 días entre el 10 de junio y el 10 de noviembre
No deberían haber huecos.
A este misterio se le añade el de las 16 lineas que faltan, cuando huecos solo hay 11...

No se me ocurre qué más hacer para averiguar qué ocurre, pero a tenor de lo visto en el pivot, estoy dispuesto a prescindir de los 5 años-distrito faltante y trabajar con los que hay...

Por si acaso se quisiera iputar:

 CÓDIGO PARA IMPUTACIÓN DE VALORES

 Vamos en un primer caso a proporcionar el valor medio y en segundo caso calcular el valor en función del resto de valores.
 En el primer caso, utilizando el valor medio estaremos subestimando la varianza, reducen la varianza de la columna, sin  incrementarla.
codigo:

imp = SimpleImputer(missing_values = np.nan, strategy = "mean")
imp.fit(X)

X2 = imp.fit_transform(X)

pd.DataFrame(X2)

coment: los datos que faltaban tienen el valor medio.

# bucle para buscar

ideas:
comparar el numero de dias sin llover con el anterior
Pasar a boleano: true > 0, false = 0
hacer un groupby por año
funcion cum sum

# Bucle:


dia_secos=0
dias_sin_lluv[]
i=0

for innerdf in df.groupby('distrito', 'anho',)
for day in innerdf
for valor in dfp.rows("chuvia")
if valor == 0 then dia_secos = dia_secos +1
else 
    dias_sin_lluv[i] = dia_secos 
    i=i+1
     dia_secos = 0
print max(dias_sin_lluv)
    

Este bucle no funciona, por lo tanto, hasta recibir un poco de ayuda alr especto, vamos a contentarnos con lluvia total en esas fechas y días sin lluvia en esas fechas.

Lluvia tota: groupby sum
Dias sin lluvia: count when chubia = 0

Entrenaremos un par de modelos con estos datos, deberían ser suficientes.

Para extrapolar, habrá que obtener un nuevo data frame con equivalentes de las predictoras:
las medias climáticas anuales por distrito, 
el porcentage hipotético de ocupación por especie y 
cualquier otra predictora que hayamos metido en el modelo

Es decir, tendremos que hacer un gruop by distrito para tener las medias de todo lo demás y sobre eso usar el modelo.

In [46]:
df[df.Chuvia < 0].groupby(by=['distrito', 'year']).count()

Instante lectura  Chuvia
distrito year                          
2        2007                 1       1
6        2007                 6       6
7        2010                 1       1
11       2010                 2       2
12       2009                12      12
         2010               154     154
13       2007                12      12
15       2009                 1       1
         2010                 2       2
17       2007                 4       4
         2010               154     154

Hay que descartar distrito 12 y 17 dekl año 2010, el resto imputo valor para los días con lluvia -9999.

In [47]:
drop_indices = df[((df.distrito == 12) | (df.distrito == 17)) & (df.year == 2010)].index
df = df.drop(drop_indices)

No funcionó la estrategia de imputación (ver abajo), así que por el momento los descartamos 

In [48]:
df.dropna(inplace=True)

df = df.replace(-9999.0, np.nan)
mean_chuvia = df.groupby(by=['distrito', 'year']).rolling(3, center=True).Chuvia.mean().droplevel(0).droplevel(0)
df['Chuvia'] = df.Chuvia.fillna(mean_chuvia)


df
# Hacer con el merge un left para que elimine del otro los nulos (distritos 12 y 17, year 2010)

,Instante lectura,Chuvia,distrito,year
0,2007-06-10,0.2,1,2007
1,2007-06-11,0.2,1,2007
2,2007-06-12,2.0,1,2007
3,2007-06-13,13.0,1,2007
4,2007-06-14,5.0,1,2007
...,...,...,...,...
76472,2019-11-06,8.8,19,2019
76473,2019-11-07,4.7,19,2019
76474,2019-11-08,7.4,19,2019
76475,2019-11-09,10.8,19,2019


In [49]:
# Creo un df a partir

df_Ch = df.groupby(by = ["year", "distrito"], as_index=False)["Chuvia"].sum()    

# HAgo un pivot para ver los resultados:

print(pd.pivot_table(df_Ch,
           index='distrito',
           columns='year',
           margins=True,
           aggfunc=sum))

# Hay nans, importante más adelante como veremos

          Chuvia                                                          \
year        2007    2008    2009    2010    2011    2012    2013    2014   
distrito                                                                   
1          207.8   426.2   382.4   364.7   265.9   405.1   527.1   418.6   
2          139.2   256.8   220.8   178.5   227.2   308.3   452.1   298.3   
3          382.0   519.3   676.8   591.8   441.0   588.3   857.9   711.5   
4          163.0   261.8   331.4   239.0   354.3   383.3   400.5   502.7   
5            NaN     NaN     NaN   167.8   269.7   314.9   400.4   411.7   
6          176.8   407.8   323.6   385.1   250.7   588.3   857.9   711.5   
7          368.4   527.0   506.6   699.2   260.0   355.4   393.7   440.6   
8          109.0   201.8    92.4   313.4   206.0   222.0   287.8   413.8   
9          199.0   219.4   204.2   394.3   246.4   237.0   359.8   402.8   
10         194.8   367.2   335.0   197.4    43.8   357.2   234.9   151.5   
11         1

In [50]:
# Ahora contamos días sin lluvia
# primer creo columna boleana, 

df['diaseco'] = df['Chuvia'] == 0

# Da columna boleana, quiero numeros para sumar (se supone que true = 1, 0 =false)

df['Dias_secos'] = df["diaseco"].astype("int") 

# Ahora creo un df con el groupby del anterior, como hice con chuvia

df_Ds = df.groupby(by = ["year", "distrito"], as_index=False)["Dias_secos"].sum()    


# Hago un pivot para ver cómo fue:

print(pd.pivot_table(df_Ds,
           index='distrito',
           columns='year',
           margins=True,
           aggfunc=sum))

# Igual que el otro, con nans. 

         Dias_secos                                                          \
year           2007    2008    2009    2010    2011    2012    2013    2014   
distrito                                                                      
1             108.0    89.0    97.0   110.0    90.0    79.0    93.0    90.0   
2             100.0    88.0    89.0   105.0    85.0    74.0    97.0    91.0   
3              61.0    49.0    60.0    73.0    69.0    67.0   102.0    85.0   
4             123.0    98.0   105.0   117.0   103.0    84.0   104.0    94.0   
5               NaN     NaN     NaN   108.0    86.0    74.0    99.0    79.0   
6              88.0    96.0    97.0    99.0    90.0    67.0   102.0    85.0   
7              59.0    56.0    57.0    60.0    68.0    89.0   111.0    86.0   
8             110.0   102.0   123.0   102.0   111.0    87.0   113.0    80.0   
9             101.0   109.0   113.0   114.0    95.0    95.0   109.0    80.0   
10             71.0    81.0    77.0   123.0   102.0 

In [51]:
# Ahora los junto, probemos con un join

DF = df_Ds.join(df_Ch, on=None, how='left',
                lsuffix= ['year', 'distrito'], rsuffix= ['year', 'distrito'], sort=False)

# Problema: repite las columnas de encaje. Con el concat tb
# DF = pd.concat([df_Ds, df_Ch], axis=1)

# Por lo menos intentaremos que queden nombradas bonitas:
DF.set_axis(['year', 'distrito', 'Dias_secos', 'Year', 'Numdistrito', 'Chuvia'], 
                    axis='columns', inplace=True)
DF


,year,distrito,Dias_secos,Year,Numdistrito,Chuvia
0,2007,1,108,2007,1,207.8
1,2007,2,100,2007,2,139.2
2,2007,3,61,2007,3,382.0
3,2007,4,123,2007,4,163.0
4,2007,6,88,2007,6,176.8
...,...,...,...,...,...,...
235,2019,15,108,2019,15,287.5
236,2019,16,100,2019,16,342.0
237,2019,17,104,2019,17,435.8
238,2019,18,102,2019,18,712.2


In [52]:

# df_qe_es = df_qe_es.drop([df_qe_es.index[4], df_qe_es.index[23], df_qe_es.index[42],
#              df_qe_es.index[204], df_qe_es.index[223]])
# df_qe_es

# Ojo: al borrar así solo se puede ejecutar una vez esta linea, 
# porque si no me las irá borrando cada vez 

# No hubo manera de borrar las columnas repes con:
# DF = DF.drop(DF.columns[[0, 1]], axis=1)



DF= df_qe_es.merge(DF, how= 'inner', right_on= ['Year', 'Numdistrito'], left_on= ['año', 'num_distr'])
DF

,DISTRITO,num_distr,año,ha_man_comun,ha_monte_particular,% ocup medio coniferas,% ocup medio frond_peren,% ocup medio cadicifolias,% ocup hipotetico coniferas,% ocup hipotetico frond_peren,% ocup hipotetico cadicifolias,ha_quemadas,Ha_tot,Prop_quem,year,distrito,Dias_secos,Year,Numdistrito,Chuvia
0,Ferrol,1,2007,4952.62,85072,5,65,10,5,5,70,106.80,90024.62,0.001186,2007,1,108,2007,1,207.8
1,Bergantiños - Mariñas Coruñesas,2,2007,3145.99,90980,27,28,16,3,3,65,126.30,94125.99,0.001342,2007,2,100,2007,2,139.2
2,Santiago - Meseta Interior,3,2007,3919.24,96731,22,32,24,4,4,70,185.40,100650.24,0.001842,2007,3,61,2007,3,382.0
3,Barbanza,4,2007,24703.18,73426,33,29,9,2,2,68,29.90,98129.18,0.000305,2007,4,123,2007,4,163.0
4,A Mariña Lucense,6,2007,19819.77,77478,12,69,7,4,4,80,30.30,97297.77,0.000311,2007,6,88,2007,6,176.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,A Limia,15,2019,58467.10,52163,20,2,21,4,2,37,333.43,110630.10,0.003014,2019,15,108,2019,15,287.5
236,Deza - Tabeirós,16,2019,16179.94,44372,24,12,34,5,5,50,262.24,60551.94,0.004331,2019,16,100,2019,16,342.0
237,O Condado - A Paradanta,17,2019,31011.84,34448,28,42,17,6,6,75,68.51,65459.84,0.001047,2019,17,104,2019,17,435.8
238,Vigo - Baixo Miño,18,2019,37737.61,33827,19,34,13,4,4,58,33.57,71564.61,0.000469,2019,18,102,2019,18,712.2


In [53]:

DF = DF.drop(['distrito', 'year'], axis = 1)
DF.head(3)




,DISTRITO,num_distr,año,ha_man_comun,ha_monte_particular,% ocup medio coniferas,% ocup medio frond_peren,% ocup medio cadicifolias,% ocup hipotetico coniferas,% ocup hipotetico frond_peren,% ocup hipotetico cadicifolias,ha_quemadas,Ha_tot,Prop_quem,Dias_secos,Year,Numdistrito,Chuvia
0,Ferrol,1,2007,4952.62,85072,5,65,10,5,5,70,106.8,90024.62,0.001186,108,2007,1,207.8
1,Bergantiños - Mariñas Coruñesas,2,2007,3145.99,90980,27,28,16,3,3,65,126.3,94125.99,0.001342,100,2007,2,139.2
2,Santiago - Meseta Interior,3,2007,3919.24,96731,22,32,24,4,4,70,185.4,100650.24,0.001842,61,2007,3,382.0


In [54]:
DF.to_pickle('DF_anho_distrito.pickle')
